In [3]:
import re
import csv
from pathlib import Path
import pandas as pd
from datasets import load_dataset

## Load metadata dataset

In [4]:
dataset = load_dataset("arxiv_dataset", data_dir='./data', split='train', verification_mode='no_checks')
df = dataset.to_pandas()
df.head()

/opt/homebrew/Caskroom/miniconda/base/envs/llm-env/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,update_date
0,0704.0001,Pavel Nadolsky,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",Calculation of prompt diphoton production cros...,"37 pages, 15 figures; published version","Phys.Rev.D76:013009,2007",10.1103/PhysRevD.76.013009,ANL-HEP-PR-07-12,hep-ph,None,A fully differential calculation in perturba...,2008-11-26
1,0704.0002,Louis Theran,Ileana Streinu and Louis Theran,Sparsity-certifying Graph Decompositions,To appear in Graphs and Combinatorics,None,None,None,math.CO cs.CG,http://arxiv.org/licenses/nonexclusive-distrib...,"We describe a new algorithm, the $(k,\ell)$-...",2008-12-13
2,0704.0003,Hongjun Pan,Hongjun Pan,The evolution of the Earth-Moon system based o...,"23 pages, 3 figures",None,None,None,physics.gen-ph,None,The evolution of Earth-Moon system is descri...,2008-01-13
3,0704.0004,David Callan,David Callan,A determinant of Stirling cycle numbers counts...,11 pages,None,None,None,math.CO,None,We show that a determinant of Stirling cycle...,2007-05-23
4,0704.0005,Alberto Torchinsky,Wael Abu-Shammala and Alberto Torchinsky,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,None,"Illinois J. Math. 52 (2008) no.2, 681-689",None,None,math.CA math.FA,None,In this paper we show how to compute the $\L...,2013-10-15


## Filter to PDF collection and append first page of text

In [5]:
pdf_files = list(Path("./data/pdf").glob("*.pdf"))
pdf_files.sort()

text_files = list(Path("./data/text").glob("*.txt"))
text_files.sort()

id_pattern = re.compile(r"(.+)v\d+\.[pdf|txt]")
ids = [
    id_pattern.match(pdf.name).group(1) for pdf in pdf_files
]

df_sample = df[df['id'].isin(ids)].copy()

def get_pdf_path(id):
    for pdf in pdf_files:
        if id in str(pdf.stem):
            return pdf.name
    return None

def get_text(id):
    for text in text_files:
        if id in str(text.stem):
            return text.read_text()

df_sample['src_pdf'] = df_sample.apply(lambda row: get_pdf_path(row['id']), axis=1)
df_sample['text'] = df_sample.apply(lambda row: get_text(row['id']), axis=1)
df_sample.dropna(subset = ['src_pdf'], inplace=True)
df_sample.to_csv("./data/arxiv_metadata.csv", index=False, quoting=csv.QUOTE_ALL)
df_sample.to_parquet("./data/arxiv_metadata.parquet", index=False)
print(f"number of rows: {len(df_sample)}")
df_sample.head()


number of rows: 1782


,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,update_date,src_pdf,text
1922271,2310.00014,Yong Ren,"Yong Ren, Tao Wang, Jiangyan Yi, Le Xu, Jianhu...",Fewer-token Neural Speech Codec with Time-inva...,Submitted to ICASSP 2024,None,None,None,cs.SD eess.AS,http://arxiv.org/licenses/nonexclusive-distrib...,Language model based text-to-speech (TTS) mo...,2023-10-03,2310.00014v1.pdf,FEWER-TOKEN NEURAL SPEECH CODEC WITH TIME-INVA...
1922288,2310.00031,Markus Marks,"Neehar Kondapaneni, Markus Marks, Manuel Knott...",Text-image Alignment for Diffusion-based Perce...,Project page: https://www.vision.caltech.edu/t...,None,None,None,cs.CV,http://arxiv.org/licenses/nonexclusive-distrib...,Diffusion models are generative models with ...,2023-10-06,2310.00031v1.pdf,Text-image Alignment for Diffusion-based Perce...
1922289,2310.00032,Qinghua Xu,"Qinghua Xu, Tao Yue, Shaukat Ali and Maite Arr...","Pretrain, Prompt, and Transfer: Evolving Digit...",None,None,None,None,cs.SE,http://creativecommons.org/licenses/by/4.0/,"Cyber-Physical Systems (CPSs), e.g., elevato...",2023-10-06,2310.00032v2.pdf,"PRETRAIN, PROMPT, AND TRANSFER: EVOLVING DIGIT..."
1922291,2310.00034,Yuzhang Shang,"Yuzhang Shang, Zhihang Yuan, Qiang Wu, Zhen Dong",PB-LLM: Partially Binarized Large Language Models,Frist work using network binarization for larg...,None,None,None,cs.LG cs.AI cs.CL,http://arxiv.org/licenses/nonexclusive-distrib...,"This paper explores network binarization, a ...",2023-10-03,2310.00034v1.pdf,PB-LLM: PARTIALLY BINARIZED LARGE LANGUAGE\nMO...
1922292,2310.00035,Xi Wang,"Xi Wang, Laurence Aitchison, Maja Rudolph",LoRA ensembles for large language model fine-t...,Update the title in the PDF file,None,None,None,cs.LG cs.AI,http://creativecommons.org/licenses/by/4.0/,Finetuned LLMs often exhibit poor uncertaint...,2023-10-06,2310.00035v1.pdf,Preprint. Under review\nENSEMBLE OF LOW-RANK A...
